In [94]:
import pandas as pd

### Objective

- The problem is technically a time-series problem because we need to base future loans based off of present loans.
- The goal is to look at a businesses' history and predict the performance of their current loans.

### Potential performance measures
- We can calculate a score based on all of the loans that a business has taken out in the past. There can also be a score for the frequency of payments made and will penalise delayed payments.
- We can also have a performance metric based on several components:
  - Use a classification model with binary cross entropy to predict if a business is paid will pay back on time or not (`probability of paying back on time`). This requires the historical information for each business.
  - Multiply the value above by the `potential yield` to get a score for the loans performance.
### Other notes
- There are some unusual decisions in the data. For example, business 343013 was declined for fradulent transactions but then had a loan approved later which does not seem correct.

# Exploratory Data Analysis (EDA)
### Drop unhelpful features
The justification for dropping each column is as follows:
- `acquisition_channel`: this field has no relation to whether a company pays a loan or not.
- `credit_officer_id`: the credit officer has no bearing on whether the company pays
- `loan_number`: this is a redundant field since we have applying_for_loan_number
- `cash_yield_15_dpd`: this field is redundant since it can be derived from total_owing_at_issue and total_recovered_15_dpd. (I am keeping this field in just to demonstrate a point later)
- `loan_id`: identifiers are not necessary for the ML model.
- `approval_status`: only working with approved loans. All loans in Due Dilligence are approved.

In [95]:
train_df = pd.read_csv('train_loan_data.csv')
train_df = train_df[train_df['approval_status'] != 'Cancelled'] # Removed cancelled loans. No money is released.
train_df = train_df[train_df['approval_status'] != 'Expired'] # Removed expired loans. No money is released.
train_df = train_df[train_df['approval_status'] != 'Declined'] # Removed declined loans. Need to predict the performance of an approved loan and not a declined loan.
train_df = train_df.drop(['acquisition_channel', 'credit_officer_id', 'loan_number', 'loan_id', 'approval_status'], axis=1)
train_df = train_df.astype({'sector' : 'category', 'dismissal_description' : 'category'}) # Convert to category

### Remove Duplicates
There seems to be an issue in the provided data since it contains duplicate entries. Here these are removed

In [96]:
print(f'Number of rows before dropping duplicates: {len(train_df)}')
train_df = train_df.drop_duplicates()
print(f'Number of rows after dropping duplicates: {len(train_df)}')

Number of rows before dropping duplicates: 23938
Number of rows after dropping duplicates: 22307


### Convert the amounts recovered into percentages

In [97]:
train_df['percentage_recovered_on_time'] = train_df['total_recovered_on_time'] / train_df['total_owing_at_issue'] 
train_df['percentage_recovered_on_time_15_dpd'] = train_df['total_recovered_15_dpd'] / train_df['total_owing_at_issue'] 
train_df = train_df.drop(['total_recovered_on_time'], axis=1)
train_df = train_df.drop(['total_recovered_15_dpd'], axis=1)

### Unbalanced dataset
The training data is too unbalanced to learn anything meaningful.
- As seen below 96.9% of the loans are paid back in full and on time
- Only 1.3% of loans are not paid back at all on time
- Only 0.08% of loans end up not making a profit (i.e. Numida loses money)

In [98]:
print(f'The percentage of loans paid back in full and on time {train_df['percentage_recovered_on_time'].value_counts()[1] / len(train_df['percentage_recovered_on_time']) }')
print(f'The percentage of loans not paid back at all {train_df['percentage_recovered_on_time'].value_counts()[0] / len(train_df['percentage_recovered_on_time'])}')
print(f'The percentage of loans where Numida loses money: {(train_df['cash_yield_15_dpd'] < 0).sum() / len(train_df['cash_yield_15_dpd'])}') # Here we can see the vast majority of clients pay back. 

The percentage of loans paid back in full and on time 0.9699197561303626
The percentage of loans not paid back at all 0.013717667100013448
The percentage of loans where Numida loses money: 0.0008517505715694625


### Further inbalance
The data are even further inbalanced if we look at how much of the loan is made back within 15 days after issue.
- 99.8% of the loans are paid back in full within 15 days of loan due date
- 0.01% of loans are not paid back at all within 15 days of loan due date

In [99]:
print(f'The percentage of loans paid back in full and on time 15 dpd {train_df['percentage_recovered_on_time_15_dpd'].value_counts()[1] / len(train_df['percentage_recovered_on_time_15_dpd']) }')
print(f'The percentage of loans not paid back at all 15 dpd {train_df['percentage_recovered_on_time_15_dpd'].value_counts()[0] / len(train_df['percentage_recovered_on_time_15_dpd'])}')

The percentage of loans paid back in full and on time 15 dpd 0.9988792755637244
The percentage of loans not paid back at all 15 dpd 0.00013448693235307301


# Training Data Conclusion
There is no point in training a model with such an extreme data imbalance. The businesses provided are good clients that are mostly expected to pay according to the train dataset. As specified in the README, Numida will likely offer loans to over 99.8% of these businesses since they pay back within 15 days. Now when we examine the test data below, there is an unexpected result since suddenly there is a much higher percentage of these businesses that are not paying back.

## Looking at the Test Data
Note this is generally bad practice to even look at the test data. But I consider the above exploratory data analysis to be conclusive enough to not proceed with training a model. So this test set exercise is just for interest's sake. 

In [100]:
test_df = pd.read_csv('test_loan_data.csv')
test_df = test_df[['total_owing_at_issue', 'total_recovered_on_time', 'total_recovered_15_dpd', 'cash_yield_15_dpd']]
test_df['percentage_recovered_on_time'] = test_df['total_recovered_on_time'] / test_df['total_owing_at_issue'] 
test_df['percentage_recovered_on_time_15_dpd'] = test_df['total_recovered_15_dpd'] / test_df['total_owing_at_issue'] 
test_df = test_df.drop(['total_recovered_on_time'], axis=1)
test_df = test_df.drop(['total_recovered_15_dpd'], axis=1)

### As seen below, the test set has an unexpected increase in loses. It would be unreasonable to expect a model trained on the training data to function correctly on the test set. 
- In the test set 9% of loans are not paid back at all 
- In the test set 8.35% of loans lose money for Numida. This is in stark contrast to 0.08% of loans in the training data.

In [101]:
print(f'The percentage of test dataset loans paid back in full and on time {test_df['percentage_recovered_on_time'].value_counts()[1] / len(test_df['percentage_recovered_on_time']) }')
print(f'The percentage of test dataset loans not paid back at all {test_df['percentage_recovered_on_time'].value_counts()[0] / len(test_df['percentage_recovered_on_time'])}')
print(f'The percentage of test dataset loans where Numida loses money: {(test_df['cash_yield_15_dpd'] < 0).sum() / len(test_df['cash_yield_15_dpd'])*100}')

The percentage of test dataset loans paid back in full and on time 0.87
The percentage of test dataset loans not paid back at all 0.0975
The percentage of test dataset loans where Numida loses money: 8.35
